# Occupancy Forecasting Notebook

In [463]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.model_selection import train_test_split, KFold

from _dfguru import DataFrameGuru as DFG
from _preprocessing import PLCount

# 1.) Data Preprocessing

In [464]:
#Reading the dataset
dfg = DFG()

data = dfg.read_data(
    path_repo="../data/cleaned_data", 
    file_name="frequency_data", 
)

data = dfg.clean_signal_data(data)

min_timestamp = data["datetime"].min().replace(hour=0, minute=0, second=0, microsecond=0)
max_timestamp = data["datetime"].max().replace(hour=0, minute=0, second=0, microsecond=0) + DateOffset(days=1)

### Once more a preprocessing run I forgot to clean the signal outside lecture time 22:00-7:30

In [465]:
course_dates_data = dfg.read_data(
        path_repo="../data/cleaned_data", 
        file_name="course_dates", 
    )


course_info_data = dfg.read_data(
    path_repo="../data/cleaned_data", 
    file_name="course_info", 
)


### 1.1) Filter out Facility Management Events

Maybe a course is missing: 8.4. - 28.6.

## Check again after preprocessing is final

HS18:
* 22.4 17:15-18:00 - safe something missing
* 26.4. 8:30-10:00 - safe something missing
* 3.5. 12:00 bis 18:30 - there is something happening -> plot but it doesn't seem to be a lecture
* 15.5. 15:15 bis 17:45 - there is also something happening 
* 4.6. 17:00 - 19:00 -> There is also something happening
* 20.6. 15:30 - 17:00, 17:15-18:45

HS19:
* 12.4. 13:45 - 15:45 -> no normal lecture is something different
* 13.5. 13:45 - 15:15, afterwards - 19:05 (something different)  -> check preprocessing! -> 5 events ruined that afternoon
* 15.5. 15:30 - 17:30 -> safe a lecture
* 5.6. 8:30 - 10:00 -> student preparing for exam
* 26.6. 8:30 - 10:00, 12:00-14:30 -> something happening!

#### Try to clean events outside lecture hours -> show that sometimes Kusss seems to be incorrect:


Visually proof that something might be off! At the times above

In [486]:
course_dates_data = dfg.derive_day(course_dates_data, "start_time")
course_dates_data_reduced = dfg.filter_by_daterange(course_dates_data, "start_time", min_timestamp.date(), max_timestamp.date())

lecture_hours = []
min_times = []
max_times = []
for grouping, df in course_dates_data_reduced.groupby(["room_id", "day"]):
    
    max_time = (df["end_time"].max() + DateOffset(minutes=15)).time()
    min_time = (df["start_time"].min() - DateOffset(minutes=15)).time()
    min_times.append(min_time)
    max_times.append(max_time)

    group_mask = (data["room_id"] == grouping[0]) & (data["datetime"].dt.date == grouping[1])
    data_group = data[group_mask]
    
    #if (df["start_time"].min().date() == datetime.date(datetime(2024, 5, 15))) and grouping[0] == 0:
    #    print(grouping)
    #    print(df.sort_values(by="start_time").to_string())
    #    print(min_time, max_time)
    #    print(data_group.to_string())
    #    print("-------------------")
    #    print()
        
    group_mask = (data["room_id"] == grouping[0]) & (data["datetime"].dt.date == grouping[1])
    data_group = data[group_mask]
    
    
    outside_hours_mask = (data_group["datetime"].dt.time < min_time) | (data_group["datetime"].dt.time > max_time)
    #print(data_group[outside_hours_mask].to_string())
    #lecture_hours = group_mask & (data["datetime"].dt.time >= min_time) & (data["datetime"].dt.time <= max_time)
    
    

## 1. Data Cleaning with PLCount

In [467]:

frequency = "1h"
plcount = PLCount()

data_dict = {}
for room_id, df in data.groupby("room_id"):
    data_filterd_room = dfg.filter_by_roomid(data, room_id)
    occ_list = plcount.run_on_whole_dataset(data_filterd_room, dfg, frequency)
    data_dict[room_id] = pd.concat(occ_list).drop_duplicates(subset="datetime").reset_index(drop=True)

100%|██████████| 82/82 [00:00<00:00, 122.46it/s]


In [468]:
#occupancy_data_cleaned = pd.concat(occ_list).drop_duplicates(subset="datetime")
#occupancy_data_raw = dfg.calc_occupancy_count(data_filterd_room, "datetime", frequency)

### 1.1 Plot differences

In [469]:
## raw_data
#fig = go.Figure()

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_raw["datetime"], 
#        y=occupancy_data_raw["CC"],
#        mode="lines",
#        name="Raw Occupancy"))

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_cleaned["datetime"], 
#        y=occupancy_data_cleaned["CC"],
#        mode="lines",
#        name="Reset at Midnight"))

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_cleaned["datetime"], 
#        y=occupancy_data_cleaned["CC_estimates"],
#        mode="lines",
#        name="PLCount Estimates"))

#fig.show()

## 2. Contextual features

Classic Contextual features: (mostly endogenous)
* Sangogboye, Arendt: Dayname, Daytype(weekend/weekday), Season(Winter, Summer), Holiday(yes/no)(exogenous)
* Sangogboye, Kjaaergard: Dayname, Daytype, Season, Holiday
* Araz Ashouri, Member, IEEE, Guy R. Newsham: Hour of Day (Sin + Cosine), Day of week (0 = Holiday, Weekends; 1 = Weekday) 

My ideas:
* Include Zwickeltag - exogenous
* Make use of lecture schedule from Kusss - exogenous



In [470]:
import torch
from torch.utils.data import DataLoader, Dataset

seed = 42
rng = np.random.default_rng(seed=seed)
torch.manual_seed(seed)

In [471]:
class OccFeatureEngineer():
    
    course_features = {"exam", "lecture"}
    datetime_features = {"weekday"}
    permissible_features = course_features.union(datetime_features)
        
    def __init__(self, cleaned_occ_data, course_dates_data, course_info_data, dfguru):

        self.occ_time_series = cleaned_occ_data
        min_timestamp = data["datetime"].min().replace(hour=0, minute=0, second=0, microsecond=0)
        max_timestamp = data["datetime"].max().replace(hour=0, minute=0, second=0, microsecond=0) + DateOffset(days=1)
        
        self.course_dates_table = dfguru.filter_by_timestamp(
            dataframe = course_dates_data,
            start_time = min_timestamp,
            end_time = max_timestamp,
            time_column = "start_time"
        )

        self.course_info_table = course_info_data
        
    
    def derive_features(self, features, room_id=None):
        
        # get the course number
        occ_time_series = self.occ_time_series.copy(deep=True)

        # check if features are permissible
        feature_set = set(features)
        set_diff = feature_set.difference(self.permissible_features)
        if set_diff:
            raise ValueError(f"Features {set_diff} are not permissible.")
        
        # check if features are already present
        feature_set = feature_set.difference(occ_time_series.columns)

        # add course features
        course_features = self.course_features.intersection(feature_set)
        if course_features:
            occ_time_series = self.add_course_features(occ_time_series, course_features, room_id)
        
        # add datetime features
        datetime_features = self.datetime_features.intersection(feature_set)
        if datetime_features:
            occ_time_series = self.add_datetime_features(occ_time_series, datetime_features)

        return occ_time_series
    
    def add_course_features(self, time_series, features, room_id):

        # room_id must be not None
        if room_id is None:
            raise ValueError("Room ID must be provided.")
        
        course_dates_in_room = dfg.filter_by_roomid(self.course_dates_table, room_id)
        
        # initialize all features to 0
        time_series["course_number"] = 0
        for feature in features:
            time_series[feature] = 0
            
        for grouping, sub_df in course_dates_in_room.groupby(["start_time", "end_time"]):
            
            # get course time_span
            course_time_mask = (time_series["datetime"] >= grouping[0]) & (time_series["datetime"] <= grouping[1])
            
            # get course_number
            time_series.loc[course_time_mask, "course_number"] = ",".join(sub_df["course_number"].values)
            
            # get course features
            if "exam" in features:
                time_series.loc[course_time_mask, "exam"] = int(sub_df["exam"].values[0])
                
            if "lecture" in features:
                time_series.loc[course_time_mask, "lecture"] = 1
        
        return time_series
            

In [472]:
for room_id in data_dict:
    
    print(f"Room ID: {room_id}")
    
    occ_time_series = OccFeatureEngineer(
        data_dict[room_id], 
        course_dates_data, 
        course_info_data, dfg
    ).derive_features(
        features=["exam", "lecture"], 
        room_id=room_id
    )      
    print("--------------------")
    print()
    
    data_dict[room_id] = occ_time_series

/tmp/ipykernel_23525/1824515244.py:68: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '258.410' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



Room ID: 0


/tmp/ipykernel_23525/1824515244.py:68: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '238.302' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



--------------------

Room ID: 1
--------------------



## 3. Validation Strategy and Train-Test Split

In [473]:
class OccupancyDataset(Dataset):
    
    def __init__(self, time_series_dict: dict, frequency:str, x_size:int=24, y_size:int=24, verbose:bool=True):
        """ Constructor for the occupancy dataset
        Task: Convert the cleaned data into a list of samples
        """
        super().__init__()
        
        # the time series must be structured as a dictionary with the room_id as the key
        self.time_series_dict = time_series_dict
        self.room_ids = list(time_series_dict.keys())
        
        # convert frequency to timedelta
        td_freq = pd.to_timedelta(frequency)
        
        # window size is 1 day
        self.x_size = x_size
        self.y_size = y_size
        
        self.verbose = verbose
        

        #print("############## Dataset Summary ##############")
            
        self.samples = []
        for room_id in self.room_ids:
            
            #print("Sample Generation for Room ID: ", room_id)
            occ_time_series = self.time_series_dict[room_id]
            
            # check for holes in the time series -> they break the rolling window
            ts_diff = occ_time_series["datetime"].diff()
            holes = occ_time_series[ts_diff > td_freq]
            
            if holes.empty:
                #print("Check 1: No holes found in the time series.")
                info, X, y = self.create_samples(occ_time_series, room_id)
                self.samples.extend(list(zip(info, X, y)))
                #print( "-----------------------------------")
                
            else:
                #print("Check 1: Holes found and taken care of.")
                
                cur_idx = 0
                for hole_idx in holes.index:
                    info, X, y = self.create_samples(occ_time_series.iloc[cur_idx:hole_idx], room_id)
                    self.samples.extend(list(zip(info, X, y)))
                    cur_idx = hole_idx
                
                # add the last part of the time series
                info, X, y = self.create_samples(occ_time_series.iloc[cur_idx:], room_id)
                self.samples.extend(list(zip(info, X, y)))
                #print( "-----------------------------------")
        #print()
                    
    def create_samples(self, time_series, room_id):
        
        occ_time_series = time_series.copy(deep=True)
        occ_time_series["day"] = occ_time_series["datetime"].dt.date
        
        X_list = []
        y_list = []
        sample_info = []
    
        # we want to predict the next N steps based on the previous T steps
        window_size = self.x_size + self.y_size
        for window in occ_time_series.rolling(window=window_size):
            # skip the first window_size elements only consider full windows
            
            X_df = window.iloc[:self.x_size]
            X = torch.Tensor(X_df["CC_estimates"].values)
            
            y_df = window.iloc[self.x_size:]
            y = torch.Tensor(y_df["CC_estimates"].values)

            X_list.append(X)
            y_list.append(y)
            
            sample_info.append((room_id, X_df["datetime"], y_df["datetime"]))

        sanity_check_1 = [len(x)==self.x_size for x in X_list[window_size-1:]]
        sanity_check_2 = [len(y)==self.y_size for y in y_list[window_size-1:]]
        
        if (all(sanity_check_1) & all(sanity_check_2)):
            #print("Check 2: All the samples have the correct size.")
            return sample_info[window_size-1:], X_list[window_size-1:], y_list[window_size-1:]
        
        else:
            raise ValueError("Sanity Check Failed")
        
        
        for day, ts_day in list(occ_time_series.groupby("day"))[:-2]:
            
            next_day = occ_time_series[occ_time_series["day"] == (day + DateOffset(days=1)).date()]

            #######################################################
            ############### Sample Generation #####################
            #######################################################
            
            X = torch.IntTensor(ts_day["CC_estimates"].values)
            y = torch.IntTensor(next_day["CC_estimates"].values)
            
            #######################################################
            #######################################################
            #######################################################
            
            x_time = ts_day["datetime"].values
            y_time = next_day["datetime"].values
            
            X_list.append(X)
            y_list.append(y)
            sample_info.append((room_id, day, x_time, y_time))
            
        return sample_info, X_list, y_list
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]


def train_val_test_split(data_dict, verbose=True):
    
    # randomly exclude chunks of the data
    train_dict = {}
    val_dict = {}
    test_dict = {}
    
    total_size = 0
    val_size = 0
    test_size = 0
    train_size = 0
    
    for room_id in data_dict:

        occ_time_series = data_dict[room_id]
        total_size += len(occ_time_series)
        
        # generate chunks with size 0.05 of the data
        index_shift = int(len(occ_time_series) * 0.05)
        indices = np.arange(0, len(occ_time_series), index_shift)
        if (len(occ_time_series)-indices[-1]) < index_shift:
            indices = indices[:-1]
            
        rng.shuffle(indices)
        
        val_indices = indices[:2]
        test_indices = indices[2 : 4]
        train_indices = indices[4:]
        
        ts_val = occ_time_series.iloc[np.array([np.arange(x, x+index_shift) for x in val_indices]).flatten()].sort_values(by="datetime").reset_index(drop=True)
        ts_test = occ_time_series.iloc[np.array([np.arange(x, x+index_shift) for x in test_indices]).flatten()].sort_values(by="datetime").reset_index(drop=True)
        ts_train = occ_time_series.iloc[np.array([np.arange(x, x+index_shift) for x in train_indices]).flatten()].sort_values(by="datetime").reset_index(drop=True)
        
        val_size += len(ts_val)
        test_size += len(ts_test)
        train_size += len(ts_train)
        
        train_dict[room_id] = ts_train
        val_dict[room_id] = ts_val
        test_dict[room_id] = ts_test
    
    if verbose:
        print("############## Split Summary ##############")
        print("Size of Validationset:", val_size/total_size)
        print("Size of Testset:", test_size/total_size)
        print("Size of Trainset:", train_size/total_size)
        print()
    
    return train_dict, val_dict, test_dict
        


## 4. Training

In [474]:
class OccupancyDenseNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, **kwargs):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        
        x = x.view(-1, self.input_size)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    


In [475]:
class MasterTrainer:
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_updates = 0
    val_interval = 250
    
    def __init__(self, model_class:nn.Module, optimizer:Optimizer, hyperparameters:dict, criterion) -> None:
        
        self.model_class = model_class
        self.criterion = criterion
        self.optimizer = optimizer
        self.hyperparameters = hyperparameters
        
    def reset_n_updates(self) -> None:
        self.n_updates = 0

    def update_hyperparameters(self, hyperparameters:dict) -> None:
        self.hyperparameters.update(hyperparameters)
        
    def custom_collate(self, x):
        info = [x[0] for x in x]
        X = torch.stack([x[1] for x in x])
        y = torch.stack([x[2] for x in x])
        return info, X, y
      
    ######## Initialization ########
    def initialize_model(self) -> nn.Module:
        model = self.model_class(**self.hyperparameters)
        return model.to(self.device)    
    
    def initialize_optimizer(self, model:nn.Module) -> Optimizer:
        
        if self.optimizer == torch.optim.Adam:
            optimizer = self.optimizer(model.parameters(), lr=self.hyperparameters["lr"])
        elif self.optimizer == torch.optim.SGD:
            optimizer = self.optimizer(model.parameters(), lr=self.hyperparameters["lr"], momentum=self.hyperparameters["momentum"])
        else:
            raise ValueError("Optimizer not supported.")
        
        return optimizer
    
    def initialize_dataset(self, train_dict:dict, val_dict:dict, test_dict:dict, frequency:str):
        
        train_set = OccupancyDataset(train_dict, frequency, self.hyperparameters["x_size"], self.hyperparameters["y_size"])
        val_set = OccupancyDataset(val_dict, frequency, self.hyperparameters["x_size"], self.hyperparameters["y_size"])
        test_set = OccupancyDataset(test_dict, frequency, self.hyperparameters["x_size"], self.hyperparameters["y_size"])
        
        return train_set, val_set, test_set
        
    def initialize_dataloader(self, train_set:Dataset, val_set:Dataset, test_set:Dataset):
        train_loader = DataLoader(train_set, batch_size=self.hyperparameters["batch_size"], shuffle=True, collate_fn=self.custom_collate)
        val_loader = DataLoader(val_set, batch_size=self.hyperparameters["batch_size"], shuffle=True, collate_fn=self.custom_collate)
        test_loader = DataLoader(test_set, batch_size=self.hyperparameters["batch_size"], shuffle=True, collate_fn=self.custom_collate)
        return train_loader, val_loader, test_loader
        
    ######## Training ########
    def train(self, dataloader:DataLoader, model:nn.Module, optimizer:Optimizer, val_dataloader:DataLoader=None):
        """
        trains a model for one epoch
        """
        
        train_loss = []
        val_loss = []
        
        for info, X, y in dataloader:
            
            optimizer.zero_grad()
        
            X = X.to(self.device)
            y = y.to(self.device)
            
            model_output = model(X)
            loss = self.criterion(model_output, y)
            
            loss.backward()
            optimizer.step()
            
            self.n_updates += 1
            
            # validate every 100 updates
            if (self.n_updates % self.val_interval == 0) and val_dataloader:
                val_losses = self.validate(val_dataloader, model)
                val_loss.append(np.mean(val_losses))
                
            train_loss.append(loss.cpu().detach())
            
        return train_loss, val_loss
    
    def validate(self, dataloader:DataLoader, model:nn.Module):
        """
        validates a model for one epoch
        """
        
        val_loss = []
        predictions = []
        with torch.no_grad():
            for info, X, y in dataloader:
                
                X = X.to(self.device)
                y = y.to(self.device)
                
                model_output = model(X)
                loss = self.criterion(model_output, y)
                
                val_loss.append(loss.cpu().detach())
                predictions.append(model_output.cpu().detach())
                
        return val_loss

In [476]:
from torch import nn
from torch.optim import Optimizer
from torch.optim import Adam

In [480]:
hyperparameters = {
        "lr": 0.001,
        "batch_size": 16,
        "hidden_size": 32,
        "x_size": 24,
        "y_size": 24,
}

mt = MasterTrainer(
    model_class = OccupancyDenseNet,
    optimizer = Adam,
    hyperparameters = hyperparameters,
    criterion= nn.MSELoss()
)

train_dict, val_dict, test_dict = train_val_test_split(data_dict, verbose=False)
train_set, val_set, test_set = mt.initialize_dataset(train_dict, val_dict, test_dict, frequency)
train_loader, val_loader, test_loader = mt.initialize_dataloader(train_set, val_set, test_set)

mt.update_hyperparameters(
    {"input_size": hyperparameters["x_size"], 
    "output_size": hyperparameters["y_size"]}
)
model = mt.initialize_model()
optimizer = mt.initialize_optimizer(model)

n_epochs = 15

mean_train_losses = []
mean_val_losses = []
val_losses = []
train_losses = []

for n_epoch in range(n_epochs):

    train_loss, val_loss = mt.train(train_loader, model, optimizer, val_loader)
        
    mean_train_losses.append(np.mean(train_loss))
    mean_val_losses.append(np.mean(val_loss))
    val_losses.extend(val_loss)
    train_losses.extend(train_loss)
    
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(1, n_epochs+1), 
        y=mean_train_losses,
        mode="lines",
        name=f"Train Loss"))

/home/berni/anaconda3/envs/webapp/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/home/berni/anaconda3/envs/webapp/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide



In [405]:
# write a nice training and testing suite


#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#n_epochs = 10
#n_updates = 0

#mean_train_losses = []
#mean_val_losses = []
#val_losses = []
#train_losses = []

#for n_epoch in range(n_epochs):

#    train_loss, val_loss = mt.train(train_loader, model, optimizer, val_loader)
        
#    mean_train_losses.append(np.mean(train_loss))
#    mean_val_losses.append(np.mean(val_loss))
#    val_losses.extend(val_loss)
#    train_losses.extend(train_loss)
    
    

#fig = go.Figure()

#fig.add_trace(
#    go.Scatter(
#        x=np.arange(1, n_epochs+1), 
#        y=mean_train_losses,
#        mode="lines",
#        name=f"Train Loss"))

#fig.add_trace(
#go.Scatter(
#    x=np.arange(1, n_epochs+1), 
#    y=mean_val_losses,
#    mode="lines",
#    name=f"Validation Loss"))   

AttributeError: 'torch.device' object has no attribute '_apply'

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

batch_size = 4
hidden_size = 24
learning_rate = 0.01


train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

model = OccupancyDenseNet(input_size=train_set.x_size, 
                          hidden_size=hidden_size, 
                          output_size=train_set.y_size).to(device)

mse = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_epochs = 10
n_updates = 0
epoch_losses = np.zeros((n_epochs, 2))

for n_epoch in range(n_epochs):

    val_loss = []
    train_loss = []
    for info, X, y in train_loader:
        
        optimizer.zero_grad()
        
        X = X.to(device)
        y = y.to(device)
        
        model_output = model(X)
        loss = mse(model_output, y)
        
        loss.backward()
        optimizer.step()
        n_updates += 1
        
        # validate every 100 updates
        train_loss.append(loss.cpu().detach())
        
    epoch_losses[n_epoch, 0] = np.mean(train_loss)
    

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(1, n_epochs+1), 
        y=epoch_losses[:, 0],
        mode="lines",
        name=f"Train Loss"))

In [ ]:
def evaluate_model(model, loader, loss):
    pass 

In [ ]:
# raw_data
fig = go.Figure()

room_id = 0

fig.add_trace(
    go.Scatter(
        x=train_dict[0]["datetime"], 
        y=train_dict[0]["CC_estimates"],
        mode="lines",
        name=f"Trainset of Room {room_id}"))
fig.add_trace(
    go.Scatter(
        x=val_dict[0]["datetime"], 
        y=val_dict[0]["CC_estimates"],
        mode="lines",
        name=f"Validationset of Room {room_id}"))
fig.add_trace(
    go.Scatter(
        x=test_dict[0]["datetime"], 
        y=test_dict[0]["CC_estimates"],
        mode="lines",
        name=f"Testset of Room {room_id}"))

fig.show()

In [ ]:
course_info_data.head(5)

,course_number,course_name,type,kind,lecturer,semester,ects,max_students,registered_students,no_dates_total,...,level,study_area,university,curriculum,assessment_criteria,teaching_methods,language,other_information,study_subject,room_id
0,340.100,Algorithmen und Datenstrukturen 1,VO,Präsenz,Alois Ferscha,2024S,"3,00",200,194,14.0,...,B1 - Bachelor 1. Jahr,Informatik,Johannes Kepler Universität Linz,Bachelorstudium Informatik,Klausur am Semesterende,Foliengestützter Vortrag,Deutsch,https://www.pervasive.jku.at/Teaching/,Software,0
1,531.163,Analysis II,UE,Präsenz,Jan-Michael Holzinger,2024S,"2,00",25,10,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
2,531.160,Analysis II,VO,Präsenz,Friedrich Pillichshammer,2024S,"3,00",100,98,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
3,256.906,Anwendung ERP-Systeme II,KS,Präsenz,N. N.,2024S,"3,00",60,59,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
4,239.141,Behavioral Economics,KS,Präsenz,Matthias Fahn,2024S,"3,00",200,145,10.0,...,B2 - Bachelor 2. Jahr,Volkswirtschaftslehre,Johannes Kepler Universität Linz,Bachelorstudium Wirtschaftswissenschaften,Klausur,"Vortrag und Diskussion, Experimente",Englisch,NaN,Wahlfächer,0


## 3. Validation Strategy and Train-Test Split


In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

### 3.1 Start easy -> from day t-1 predict day t

In [ ]:
# bring the data into correct structure: X=day_t-1, y=day_t

samples = []
sample_info = []
for room_id in data_dict:
    occ_df = data_dict[room_id]
    occ_df["day"] = occ_df["datetime"].dt.date
    
    # exclude last day from grouping
    for day, df in list(occ_df.groupby("day"))[:-2]:

        next_day = occ_df[occ_df["day"] == (day + DateOffset(days=1)).date()]
        
        X = df["CC_estimates"].values
        y = next_day["CC_estimates"].values
        
        x_times = df["datetime"].values
        y_times = next_day["datetime"].values

        samples.append((X, y, y_times))
        sample_info.append((room_id, day))
        
samples = np.array(samples)
sample_info = np.array(sample_info)

In [ ]:
sample_info.shape

(162, 2)

In [ ]:
kfold = KFold(n_splits=5, shuffle=False)

predictions = []
for train, test in kfold.split(samples):
    X_train = samples[train][:,0]
    y_train = samples[train][:,1]
    
    test_samples =  samples[test]
    X_test = test_samples[:,0]
    y_test = test_samples[:,1]
    
    print("--------------------")
    regr = MLPRegressor(
        hidden_layer_sizes=(6,6),
        batch_size=1,
        random_state=1, 
        max_iter=5000).fit(X_train, y_train)
    score = regr.score(X_test, y_test)
    print(score)
    preds = regr.predict(X_test[:10])
    predictions.append((preds, y_test[:10], test_samples[:,2]))
    print("--------------------")

--------------------
-0.7340202309459386
--------------------
--------------------
-0.30616667394601976
--------------------
--------------------
-0.5286922465710288
--------------------
--------------------
-0.36444199859021387
--------------------
--------------------
-0.3560925020711305
--------------------


In [ ]:
fig = go.Figure()

x_axis = np.concatenate([pd.to_datetime(x)for x in predictions[0][2]])

fig.add_trace(
    go.Scatter(
        x=x_axis, 
        y=np.concatenate(predictions[0][0]),
        mode="lines",
        name="Predictions"))


fig.add_trace(
    go.Scatter(
        x=x_axis, 
        y=np.concatenate(predictions[0][1]),
        mode="lines",
        name="Target"))



fig.show()

In [ ]:
[pd.to_datetime(x)for x in predictions[0][2]]

[DatetimeIndex(['2024-04-08 00:00:00', '2024-04-08 01:00:00',
                '2024-04-08 02:00:00', '2024-04-08 03:00:00',
                '2024-04-08 04:00:00', '2024-04-08 05:00:00',
                '2024-04-08 06:00:00', '2024-04-08 07:00:00',
                '2024-04-08 08:00:00', '2024-04-08 09:00:00',
                '2024-04-08 10:00:00', '2024-04-08 11:00:00',
                '2024-04-08 12:00:00', '2024-04-08 13:00:00',
                '2024-04-08 14:00:00', '2024-04-08 15:00:00',
                '2024-04-08 16:00:00', '2024-04-08 17:00:00',
                '2024-04-08 18:00:00', '2024-04-08 19:00:00',
                '2024-04-08 20:00:00', '2024-04-08 21:00:00',
                '2024-04-08 22:00:00', '2024-04-08 23:00:00'],
               dtype='datetime64[ns]', freq=None),
 DatetimeIndex(['2024-04-09 00:00:00', '2024-04-09 01:00:00',
                '2024-04-09 02:00:00', '2024-04-09 03:00:00',
                '2024-04-09 04:00:00', '2024-04-09 05:00:00',
                '2

In [ ]:

#n_lags = 10
#lag_list = list(range(0, n_lags))

#fig = go.Figure()
#acf_array = acf(occ_df["CC_estimates"], nlags=n_lags) 
#fig.add_trace(
#    go.Bar(
#        x=lag_list,
#        y=acf_array,
#        name='Auto Correlation'
#    )
#)
#pacf_array = pacf(occ_df["CC_estimates"], nlags=n_lags) 
#fig.add_trace(
#    go.Bar(
#        x=lag_list,
#        y=pacf_array,
#        name='Partial Auto Correlation'
#    )
#)

#fig.show()
#critical_value = 1.96 / np.sqrt(len(occ_df))
#fig.add_hline(y=critical_value, line_width=1, line_dash="dot", line_color="black")
#fig.add_hline(y=-critical_value, line_width=1, line_dash="dot", line_color="black")

occ_df["CC_estimates"].corr(occ_df["lecture"])

0.7544044376991862

## 5. Basic Statistics and Plots

In [ ]:
#data_filterd = filter_room_id(data, 0)

Bar Charts

In [ ]:
# Paper: 
# Basic bar plots count of events after resampling
# Check if it follows some distribution

#data_filterd = filter_by_event_type(data_filterd, 1)
#data_resampled = resample(data_filterd, "datetime", "5min", "count")


#data_trimmed = data_resampled[data_resampled["event_type"] > 0]
#value_counts = data_trimmed.value_counts("event_type")

#fig = go.Figure()
## line plot
#fig.add_trace(
#    go.Bar(
#        x=value_counts.index, 
#        y=value_counts, 
#        name='value_counts'))


Boxplot

In [ ]:
# Boxplot plots -> Boxplot for every day and time
# Do for different events and rooms

#data_plot = occupancy_data_cleaned
#data_plot = handler.derive_time(data_plot)
#data_plot = handler.derive_weekday(data_plot)
#data_plot = handler.derive_week(data_plot)
#data_plot = data_plot.drop("datetime", axis=1)

#data_plot = data_plot.groupby(["week", "time", "weekday"]).sum().reset_index()

#print(data_plot)
#fig = px.box(data_plot, x="time", y="CC_estimates")
## make figure larger
#fig.update_layout(
#    autosize=False,
#    width=1000,
#    height=500,
#)
#fig.show()

More plots in the other notebook

## 1.Patterns

clustering -> different algorithms, k-means

Papers tried: 

### 1.1 Occupancy data analytics and prediction: A case study - Xin Liang a, b, Tianzhen Hong b, *, Geoffrey Qiping Shen

Distance Measures: Eucleadian, Dynamic Time Warp, Correlation Simmilarity -> compared among all different configurations

Clustering Algos: K-Means, 

Algorithm Evaluation: Davies-Bouldin index, to find best k

Insights of the paper:

* High variability in the data statistical methods using some kind of mean will fail miserably!

In [ ]:
import json
from collections import defaultdict
import itertools
from tqdm import tqdm


In [ ]:
def filter_by_time(data, time_column, start_time, end_time):
    
    return data[(data[time_column].dt.time >= start_time) & (data[time_column].dt.time <= end_time)]

def plot_cluster_details(cluster_number, clustering_model, samples, time_axis, plot_centers):
    
    indices = np.arange(len(samples))
    
    fig = go.Figure()
    
    cluster_indices = indices[clustering_model.labels_ == cluster_number]
    
    if plot_centers:
        cluster_center = clustering_model.cluster_centers_[cluster_number]
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=cluster_center, 
                name=f'Center',
                mode="lines+markers"        
            )
        )

    
    print(f"################ {cluster_number} ################")
    print(cluster_indices)

    for x in range(len(cluster_indices)):
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=samples[cluster_indices[x]], 
                mode="lines+markers"        
            )
        )
    
    return fig


In [ ]:
# split into daily samples
data_clustering = occupancy_data_cleaned
data_clustering = dfg.derive_time(data_clustering)
data_clustering = dfg.derive_weekday(data_clustering)
data_clustering = dfg.derive_week(data_clustering)
data_clustering = dfg.derive_day(data_clustering)

occupancy_counts = []
sample_info = []
group_list = []
times = data_clustering["time"].unique()

for group in data_clustering.groupby("day"):
    
    occupancy_counts.append(group[1]["CC_estimates"].values)
    sample_info.append(group[1][["weekday", "weekday_name", "week", "day"]].values[0])
    group_list.append(group[0])

occupancy_counts = np.array(occupancy_counts[:-1])
sample_info = np.array(sample_info[:-1])
group_list = np.array(group_list[:-1])


In [ ]:
occupancy_counts

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score, pairwise_distances

from scipy.spatial.distance import pdist, squareform



In [ ]:
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

from pyclustering.utils.metric import distance_metric, type_metric
from scipy.spatial.distance import correlation, cosine
from dtaidistance import dtw
from tslearn.metrics import dtw as dtw_tslearn
import heapq

In [ ]:
room_capacity = handler.filter_by_room_id(course_dates_data, 0)["room_capacity"].unique()

In [ ]:
import warnings
import random
np.warnings = warnings

# list of 50 random seeds between 0 and 100
seeds = list(range(0,1000))
seeds = random.sample(seeds, 100)

cluster_data = occupancy_counts

parameters = {"n_clusters":list(range(2,10)),
              "distance_metric":[
                  distance_metric(type_metric.EUCLIDEAN),
                  distance_metric(type_metric.CHEBYSHEV),
                  distance_metric(type_metric.MANHATTAN),
                  distance_metric(type_metric.EUCLIDEAN_SQUARE),
                  distance_metric(type_metric.USER_DEFINED, func=dtw),
                  distance_metric(type_metric.USER_DEFINED, func=dtw_tslearn),
                  distance_metric(type_metric.USER_DEFINED, func=correlation),
                  distance_metric(type_metric.USER_DEFINED, func=cosine),                   
              ]}

# change -> first iterate over parameters than compute avg or median score for each parameter combination over all seeds and variance

comb_iterator = ParameterSearch(parameter_dict=parameters).grid_search_iterator(tqdm_bar=True)

combinations = []
avg_scores = []
median_scores = []
std_scores = []

avg_sizes = []
for i, params in enumerate(comb_iterator):
    
    bdi_scores = []
    sizes = []
    for seed in seeds:
    
        # sklearn
        #clustering_model = KMeans(n_clusters=params["n_clusters"], random_state=seed)
        #clustering_model.fit(occupancy_counts)
        #cluster_labels = clustering_model.labels_
        
        # pyclustering
        initial_centers = kmeans_plusplus_initializer(cluster_data, params["n_clusters"]).initialize()
        #clustering_model = kmeans(cluster_data, initial_centers, distance_metric=params["distance_metric"])
        clustering_model = kmedians(cluster_data, initial_centers, distance_metric=params["distance_metric"])
        clustering_model.process()
        
        cluster_labels = np.empty(len(occupancy_counts))
        for i, cluster in enumerate(clustering_model.get_clusters()):
            cluster_labels[cluster] = i

        cluster_sizes = np.sort([len(cluster) for cluster in clustering_model.get_clusters()])
        sizes.append(cluster_sizes)
        
        score = davies_bouldin_score(occupancy_counts, cluster_labels)
        bdi_scores.append(np.round(score, 8))
        
    combinations.append((i, params))
    avg_sizes.append(np.mean(sizes, axis=0))
    print(np.mean(sizes, axis=0))

    avg_scores.append(np.mean(bdi_scores))
    median_scores.append(np.median(bdi_scores))
    std_scores.append(np.var(bdi_scores))
    

  2%|▏         | 1/64 [00:01<01:49,  1.74s/it]

[13.89 68.11]


  3%|▎         | 2/64 [00:03<02:03,  2.00s/it]

[14.19 67.81]


  5%|▍         | 3/64 [00:05<01:55,  1.90s/it]

...

[ 1.    1.03  1.31  2.05  4.23  7.46 10.05 19.52 35.35]


 98%|█████████▊| 63/64 [05:25<00:05,  5.65s/it]

[ 1.    1.02  1.38  2.18  4.28  7.98 10.35 18.61 35.2 ]


100%|██████████| 64/64 [05:31<00:00,  5.18s/it]

[ 1.    1.06  1.44  2.21  4.46  7.55 10.35 18.51 35.42]


In [ ]:
np.array(sizes)[:,0].mean()

15.48

In [ ]:
np.mean(sizes, axis=0)

array([15.48, 66.52])

In [ ]:
k = 10
bot_k = np.argsort(avg_scores)[:k]
bot_k_std= np.array(std_scores)[bot_k]
final_choice = bot_k[np.argmin(bot_k_std)]

final_choice

61

In [ ]:
def plot_cluster_centers(n_clusters, cluster_centers, times):
    
    fig = go.Figure()
    
    for i in range(n_clusters):
        # line plot
        fig.add_trace(
            go.Scatter(
                x=times, 
                y=cluster_centers[i], 
                name=f'cluster {i}',
                mode="lines+markers"        
            )
        )
        
    return fig

In [ ]:
# test optimal configuration
optimal_params = combinations[61][1]
print(optimal_params, optimal_params["distance_metric"].get_type(), optimal_params["distance_metric"].get_function())

# sklearn
#clustering_model = KMeans(**optimal_params)
#clustering_model.fit(cluster_data)
#n_clusters = len(clustering_model.cluster_centers_)

initial_centers = kmeans_plusplus_initializer(cluster_data, optimal_params["n_clusters"]).initialize()
#clustering_model = kmeans(cluster_data, initial_centers, distance_metric=optimal_params["distance_metric"])
clustering_model = kmedians(cluster_data, initial_centers, distance_metric=params["distance_metric"])
clustering_model.process()

cluster_indices = clustering_model.get_clusters()
n_clusters = len(cluster_indices)
cluster_labels = np.empty(len(cluster_data))
for i, cluster in enumerate(cluster_indices):
    cluster_labels[cluster] = i

score = davies_bouldin_score(cluster_data, cluster_labels)
            
print(f"Number of clusters: {n_clusters}")
    
#cluster_center = clustering_model.cluster_centers_[i]
#cluster_centers = clustering_model.get_means()
cluster_centers = clustering_model.get_medians()

plot_cluster_centers(n_clusters, cluster_centers, times).show()

print(f"BDI: {score}")

{'n_clusters': 9, 'distance_metric': <pyclustering.utils.metric.distance_metric object at 0x7acf4fd0f140>} 1000 <function dtw at 0x7acf60035a80>
Number of clusters: 9


BDI: 0.8087909444389905


kmeans:

{'n_clusters': 14, 'distance_metric': <pyclustering.utils.metric.distance_metric object at 0x7acf4f19fc20>}
Number of clusters: 14
BDI: 0.803127753888182


# Analyize cluster info

For inspiration of quantities to derive look at PRECount paper

In [ ]:
def filter_by_date(dataframe, time_column, date):
    return dataframe[dataframe[time_column].dt.date == date]

def number_courses(dataframe, date):
    courses_of_day = filter_by_date(dataframe, "start_time", date)
    return len(courses_of_day)

def number_exams(dataframe, date):
    courses_of_day = filter_by_date(dataframe, "start_time", date)
    return sum(courses_of_day["exam"] == 1)

def derive_weekend(dataframe, date_column):
    dataframe["weekend"] = dataframe[date_column].apply(lambda x: x.weekday()) >= 5
    return dataframe

national_holidays_2024 = [
    datetime.strptime('01/01/2024', '%m/%d/%Y').date(),  # Neujahr
    datetime.strptime('01/06/2024', '%m/%d/%Y').date(),  # Heilige Drei Könige
    datetime.strptime('04/01/2024', '%m/%d/%Y').date(),  # Ostermontag
    datetime.strptime('05/01/2024', '%m/%d/%Y').date(),  # Staatsfeiertag
    datetime.strptime('05/09/2024', '%m/%d/%Y').date(),  # Christi Himmelfahrt
    datetime.strptime('05/20/2024', '%m/%d/%Y').date(),  # Pfingstmontag
    datetime.strptime('05/30/2024', '%m/%d/%Y').date(),  # Fronleichnam
    datetime.strptime('08/15/2024', '%m/%d/%Y').date(),  # Mariä Himmelfahrt
    datetime.strptime('10/26/2024', '%m/%d/%Y').date(),  # Nationalfeiertag
    datetime.strptime('11/01/2024', '%m/%d/%Y').date(),  # Allerheiligen
    datetime.strptime('12/08/2024', '%m/%d/%Y').date(),  # Mariä Empfängnis
    datetime.strptime('12/25/2024', '%m/%d/%Y').date(),  # Christtag
    datetime.strptime('12/26/2024', '%m/%d/%Y').date(),  # Stefanitag
]

university_holidays_2024 = [
    # 18.05.2024
    datetime.strptime('05/18/2024', '%m/%d/%Y').date(),
    # 21.05.2024
    datetime.strptime('05/21/2024', '%m/%d/%Y').date(),
    # 04.05.2024
    datetime.strptime('05/04/2024', '%m/%d/%Y').date(),
    # 31.05.2024
    datetime.strptime('05/31/2024', '%m/%d/%Y').date(),
]

zwickeltage_2024 = [
    # 10.05.2024
    datetime.strptime('05/10/2024', '%m/%d/%Y').date(),
    # 31.05.2024
    datetime.strptime('05/31/2024', '%m/%d/%Y').date(),
    # 16.08.2024
    datetime.strptime('08/16/2024', '%m/%d/%Y').date(),
]

easter_break_2024 = [
    # easter break: 25.03.2024 - 06.04.2024
    datetime.strptime('03/25/2024', '%m/%d/%Y').date(),
    datetime.strptime('04/06/2024', '%m/%d/%Y').date(),]
summer_break_2024 = [
    # summer break: 1.07.2024 - 30.09.2024
    datetime.strptime('07/29/2024', '%m/%d/%Y').date(),
    datetime.strptime('09/09/2024', '%m/%d/%Y').date(),
]
def calculate_break_dates(break_dates):
    break_days = []
    num_dates = (break_dates[1] - break_dates[0]).days + 1
    for i in range(num_dates):
        break_days.append((break_dates[0] + DateOffset(days=i)).date())
        
    return break_days

easter_break_2024 = calculate_break_dates(easter_break_2024)

summer_break_2024 = calculate_break_dates(summer_break_2024)



def derive_number_exams(dataframe, date_column):
    dataframe["number_exams"] = dataframe[date_column].apply(lambda x: number_exams(course_dates_data, x))
    return dataframe

def derive_number_courses(dataframe, date_column):
    dataframe["number_courses"] = dataframe[date_column].apply(lambda x: number_courses(course_dates_data, x))
    return dataframe

def derive_holiday(dataframe, holiday_dates, date_column, out_column):
    dataframe[out_column] = dataframe[date_column].isin(holiday_dates)
    return dataframe

In [ ]:
def enrich_cluster_infos(cluster_infos):
    cluster_infos = derive_number_courses(cluster_infos, "day")
    cluster_infos = derive_number_exams(cluster_infos, "day")

    cluster_infos = derive_weekend(cluster_infos, "day")

    cluster_infos = derive_holiday(cluster_infos, zwickeltage_2024, "day", "zwickltag")
    cluster_infos = derive_holiday(cluster_infos, national_holidays_2024 + university_holidays_2024, "day", "holiday")

    cluster_infos = derive_holiday(cluster_infos, easter_break_2024 + summer_break_2024, "day", "break")
    
    return cluster_infos

In [ ]:
indices = np.arange(len(occupancy_counts))

for i in range(n_clusters):  
    
    cluster_indices = indices[cluster_labels == i]

    cluster_infos = sample_info[cluster_indices]


    cluster_infos = pd.DataFrame(cluster_infos, index=cluster_indices, columns=["weekday", "weekday_name", "week", "day"])

    cluster_infos = enrich_cluster_infos(cluster_infos)
    
    print("###################")
    print(cluster_infos.to_string())
    print("###################")
    print()

###################
   weekday weekday_name week         day  number_courses  number_exams  weekend  zwickltag  holiday  break
5        5     Saturday   15  2024-04-13               1             0     True      False    False  False
10       3     Thursday   16  2024-04-18               5             0    False      False    False  False
17       3     Thursday   17  2024-04-25               4             0    False      False    False  False
18       4       Friday   17  2024-04-26               1             1    False      False    False  False
24       3     Thursday   18  2024-05-02               4             0    False      False    False  False
28       0       Monday   19  2024-05-06               5             0    False      False    False  False
37       2    Wednesday   20  2024-05-15               5             0    False      False    False  False
38       3     Thursday   20  2024-05-16               4             0    False      False    False  False
44       2    Wed

In [ ]:
# derive the actual courses -> based on start_time 
# E.g.: if a day had a course at 8:30 or not and so on



In [ ]:
# try to cluster the results based on the stuff that happened on that day with decision tree
from sklearn.tree import DecisionTreeClassifier

# clusters are the labels
# cluster_infos are the features



Distance metrics:

* Euclidean
* Correlation Similarity
* Dynamic Time Warp



In [ ]:
# get optimal configuration

from sklearn.cluster import DBSCAN, BisectingKMeans
from sklearn.manifold import TSNE

seeds = range(0,100)

parameters = {
    "general_settings":{},
    "downprojection": False,
    "algorithm":"kmeans",
    "kmeans":{
        "n_clusters": list(range(2, 15)),
    },
    "bisectingkmeans":{
        "n_clusters": list(range(2, 15)),
        "n_init": [1, 5, 10, 20],
    },
    "affinity":{
        "damping": np.linspace(0.5, 1, 10, endpoint=False),
        "affinity": ["euclidean"],
    },
    "dbscan":{
        "eps": np.linspace(0.1, 1, 10),
        "min_samples": list(range(5, 15)),
        "metric": ["euclidean", "manhattan", "correlation"],
        "leaf_size": [2,5,10,30],
    },
    "meanshift":{
        },
    "spectral":{
       "n_clusters": list(range(2, 15)), 
    },
    "agglomerative":{
        "n_clusters": list(range(2, 15)),
        "metric": ["manhattan", "cosine", "l1", "l2"],
        "linkage": ["complete", "average", "single"],
    },
    
    "tsne":{
        "n_components": [3],
        "perplexity": [30],
    }
}


if parameters["algorithm"] == "kmeans":
    algorithm_instance = KMeans
elif parameters["algorithm"] == "bisectingkmeans":
    algorithm_instance = BisectingKMeans
elif parameters["algorithm"] == "affinity":
    algorithm_instance = AffinityPropagation
elif parameters["algorithm"] == "dbscan":
    algorithm_instance = DBSCAN
elif parameters["algorithm"] == "meanshift":
    algorithm_instance = MeanShift
elif parameters["algorithm"] == "spectral":
    algorithm_instance = SpectralClustering
elif parameters["algorithm"] == "agglomerative":
    algorithm_instance = AgglomerativeClustering
else:
    raise ValueError("Algorithm not implemented!")

argmin_list = []
for seed in seeds:
    
    bdi_scores = []
    combination = []
    comb_iterator = ParameterSearch(parameter_dict=parameters[parameters["algorithm"]]).grid_search_iterator(tqdm_bar=True)

    for i, params in enumerate(comb_iterator):
        
        #print(f"################ {n_clusters} ################")
        
        clustering_model = algorithm_instance(**params)
        
        #comb_iterator_tsne = ParameterSearch(parameter_dict=parameters["tsne"]).grid_search_iterator(tqdm_bar=False)
        
        
        #for params_tsne in comb_iterator_tsne:
        #    if parameters["downprojection"]:
        #        samples_embedded = TSNE(**params_tsne).fit_transform(samples)
            
        samples_embedded = samples
        try:
            if params["affinity"] == "precomputed":
                similarities = - pairwise_distances(samples_embedded + 1e-8, metric="manhattan")
                clustering_model.fit(similarities)
            else:
                clustering_model.fit(samples_embedded)
        except:
            clustering_model.fit(samples_embedded + 1e-8)


        #plot_cluster_centers(n_clusters, clustering_model, times[0]).show()
        if len(np.unique(clustering_model.labels_)) == 1:
            score = 1000
        else:
            score = davies_bouldin_score(samples, clustering_model.labels_)

        #print(score)
        bdi_scores.append(np.round(score, 6))
        #combination.append((params, params_tsne))
        combination.append((i, params))

        
    argmin_list.append(np.argmin(bdi_scores))
np.unique(argmin_list, return_counts=True)

100%|██████████| 13/13 [00:00<00:00, 226.09it/s]


(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([ 5,  1,  1,  2,  2,  2,  3,  6,  7, 15, 21, 35]))

### Optimal Parameters

algorithm = KMeans , INVESTIGATE FURTHER -- Read the two papers and implement their approach<br>
include_seeds = True <br>
algo_parameters = {
    "n_clusters":12
}

BisectingKMeans, INVESTIGATE FURTHER -- we need seeds<br>
{'n_clusters': 14, 'n_init': 20}

algorithm = AffinityPropagation -> works well, GRAPHCLUSTERING<br>
{'damping': 0.5, 'affinity': 'euclidean'}

AgglomerativeClustering -> works well <br>
{'n_clusters': 2, 'metric': 'manhattan', 'linkage': 'average'}

Fuzzy C-Means <br>
Implementation is flawed

DBScan <br>
Figure out better parameters


Dimensionality reduction:
* TSNE enhanced Clustering:

No TSNE<br>
{'n_clusters': 14}
Number of clusters: 14
BDI: 0.7672581861245025

* UMAP enahnced Clustering


In [ ]:
combination

[(0, {'n_clusters': 2}),
 (1, {'n_clusters': 3}),
 (2, {'n_clusters': 4}),
 (3, {'n_clusters': 5}),
 (4, {'n_clusters': 6}),
 (5, {'n_clusters': 7}),
 (6, {'n_clusters': 8}),
 (7, {'n_clusters': 9}),
 (8, {'n_clusters': 10}),
 (9, {'n_clusters': 11}),
 (10, {'n_clusters': 12}),
 (11, {'n_clusters': 13}),
 (12, {'n_clusters': 14})]

In [ ]:
# test optimal configuration
print(f"################ {parameters['algorithm']} ################")
optimal_params = combination[10][1]
print(optimal_params)

clustering_model = algorithm_instance(**optimal_params)

try:
    if params["affinity"] == "precomputed":
        similarities = - pairwise_distances(samples + 1e-8, metric="manhattan")
        clustering_model.fit(similarities)
    else:
        clustering_model.fit(samples)
except:
    clustering_model.fit(samples)

try:
    n_clusters = len(clustering_model.cluster_centers_)
except:
    try:
        clustering_model.cluster_centers_ = samples[clustering_model.cluster_centers_indices_]
        n_clusters = len(clustering_model.cluster_centers_indices_)
    except:
        n_clusters = len(np.unique(clustering_model.labels_))
    
    

print(f"Number of clusters: {n_clusters}")

#plot_cluster_centers(n_clusters, clustering_model, times[0]).show()

score = davies_bouldin_score(samples, clustering_model.labels_)
print(f"BDI: {score}")

################ kmeans ################
{'n_clusters': 12}
Number of clusters: 12
BDI: 0.809285789249472


In [ ]:
for i in range(n_clusters):  
    plot_cluster_details(i, clustering_model, samples, times[0], plot_centers=False).show()

################ 0 ################
[ 0  7 14 21]


################ 1 ################
[ 6 12 13 19 20 23 25 26 27 28 31 32 33 34 40 41 42 43 46 47 48 50 52 53
 54 55 60 61 62 64 67 68 69 74 75 76 77]


################ 2 ################
[ 5  8 10 17 18 24 37 38 44 45 58 59 66 72 73 79]


################ 3 ################
[78]


################ 4 ################
[80]


...

################ 7 ################
[22]


################ 8 ################
[ 2  9 16 30 51 57 65]


################ 9 ################
[11 35 49 56 63 70]


################ 10 ################
[3]


################ 11 ################
[ 1 15 36 71]


In [ ]:
indices = np.arange(len(samples))

fig = go.Figure()

for i in range(n_clusters):

    cluster_indices = indices[clustering_model.labels_ == i]
    cluster_center = clustering_model.cluster_centers_[i]
    
    
    cluster_group_list = group_list[cluster_indices]

    print(f"################ {i} ################")
    print(cluster_indices)
    print(cluster_group_list)
    print()
    
    # line plot
    fig.add_trace(
        go.Scatter(
            x=times[0], 
            y=cluster_center, 
            name=f'cluster {i}',
            mode="lines+markers"        
        )
    )
        
fig.show()

################ 0 ################
[ 6 12 13 19 20 23 25 26 27 31 32 33 34 40 41 42 43 46 47 48 52 53 54 55
 60 61 62 67 68 69 74 75 76]
[datetime.date(2024, 4, 14) datetime.date(2024, 4, 20)
 datetime.date(2024, 4, 21) datetime.date(2024, 4, 27)
 datetime.date(2024, 4, 28) datetime.date(2024, 5, 1)
 datetime.date(2024, 5, 3) datetime.date(2024, 5, 4)
 datetime.date(2024, 5, 5) datetime.date(2024, 5, 9)
 datetime.date(2024, 5, 10) datetime.date(2024, 5, 11)
 datetime.date(2024, 5, 12) datetime.date(2024, 5, 18)
 datetime.date(2024, 5, 19) datetime.date(2024, 5, 20)
 datetime.date(2024, 5, 21) datetime.date(2024, 5, 24)
 datetime.date(2024, 5, 25) datetime.date(2024, 5, 26)
 datetime.date(2024, 5, 30) datetime.date(2024, 5, 31)
 datetime.date(2024, 6, 1) datetime.date(2024, 6, 2)
 datetime.date(2024, 6, 7) datetime.date(2024, 6, 8)
 datetime.date(2024, 6, 9) datetime.date(2024, 6, 14)
 datetime.date(2024, 6, 15) datetime.date(2024, 6, 16)
 datetime.date(2024, 6, 21) datetime.date(2024,

## 2. Train and Test split

In [ ]:
data_filterd = filter_room_id(data, 0)
data_filterd = filter_event_type(data_filterd, 1)


#train, test = train_test_split(samples, test_size=0.1, random_state=42)

#train = np.array(train)
#test = np.array(test)

#kf = KFold(n_splits=10)

#for train_index, valid_index in kf.split(train):
    
#    train_fold = train[train_index]
#    valid_fold = train[valid_index]
    
#    break

## 3. Occupancy Forecasting

In [ ]:
# check out some of the papers in the review
# start with some traditional methods

## 4. Count Forecasting
### Try more general approach if 1. is too hard, gather literature on count forecasting